In [1]:
# global import
import os
import numpy as np
import utils
import pickle

# global variable
voxel_size = 0.1 # which means 0.1 meter


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
if os.path.exists("./data/checkpoint/voxels.pkl") and os.path.exists("./data/checkpoint/colors.pkl"):
    with open("./data/checkpoint/voxels.pkl", "rb") as f:
        voxels = pickle.load(f)
    with open("./data/checkpoint/colors.pkl", "rb") as f:
        colors = pickle.load(f)
else:
    raw_points, raw_colors = utils.las2npy("../../dataset/001-010.las")
    # rarefaction 体素滤波
    raw_points = raw_points[::50]
    raw_colors = raw_colors[::50]
    voxels, idx_o2n, idx_n2o, unique_counts = utils.voxel_downsample(raw_points, voxel_size, True)
    colors = raw_colors[idx_o2n]
    with open("./data/checkpoint/voxels.pkl", "wb") as f:
        pickle.dump(voxels, f)
    with open("./data/checkpoint/colors.pkl", "wb") as f:
        pickle.dump(colors, f)

utils.npy2ply(voxels, colors, "./data/pipeline_original.ply")

# Height filtration section

In [3]:
# 执行基于高程信息的基础滤波，这种滤波方法不能直接对整个点云集合处理，
# 因为输电线路的架设不一定维持在同一海平面，均值滤波需要分成小批次处理
start_point = 0
march_steop = int(2.5e3)
pivot_list = [] # pivot list
stapt_list = [] # start point list

boundary = voxels.shape[0]
while start_point < boundary:
    pivot = utils.get_average_pivot(voxels[start_point:min(start_point + march_steop, boundary), 2])
    pivot_list.append(pivot)
    stapt_list.append(start_point)
    start_point += march_steop

height_filter_mask = np.zeros((0, ), dtype=bool)
for stapt, pivot in zip(stapt_list, pivot_list):
    height_filter_mask = np.concatenate((height_filter_mask, voxels[stapt:min(stapt + march_steop, boundary), 2] > pivot))

voxels_filteredby_height = voxels[height_filter_mask]
colors_filteredby_height = colors[height_filter_mask]

utils.npy2ply(voxels_filteredby_height, colors_filteredby_height, "./data/pipeline_height_filtration.ply", use_txt=False)

# Power line extraction

In [5]:
# 多进程处理子空间坐标几何特征计算
from concurrent.futures import ProcessPoolExecutor, as_completed

if os.path.exists("./data/checkpoint/line_feat_mat.pkl"):
    with open("./data/checkpoint/line_feat_mat.pkl", "rb") as f:
        line_feat_mat = pickle.load(f)
else:
    num_workers = 4
    batch_size = voxels_filteredby_height.shape[0] // num_workers
    line_feat_mat_list = [None] * num_workers
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures_dict = {
            executor.submit(
                utils.eigval_radius,
                voxels_filteredby_height[i * batch_size : (i + 1) * batch_size],
                voxel_size * 75.0
            ) : i for i in range(num_workers)
        }
        for future in as_completed(futures_dict):
            feat_mat, _ = future.result()
            line_feat_mat_list[futures_dict[future]] = feat_mat
    line_feat_mat = np.concatenate(line_feat_mat_list, axis=0)
    with open("./data/checkpoint/line_feat_mat.pkl", "wb") as f:
        pickle.dump(line_feat_mat, f)

eigval progress: 100%|████████████████████████████████████| 197505/197505 [02:19<00:00, 1412.78it/s]


In [6]:
# 因为使用了多进程分组处理，所以有可能原先分组不能整除刚好分给各个进程
# 因此处理后的掩码长度也许和原点云坐标数量不一，需要按照掩码的长度调整
voxels_filteredby_height = voxels_filteredby_height[:line_feat_mat.shape[0]]
colors_filteredby_height = colors_filteredby_height[:line_feat_mat.shape[0]]

line_feat_max = np.max(line_feat_mat, axis=0)
line_feat_min = np.min(line_feat_mat, axis=0)
print(line_feat_max)
print(line_feat_min)
line_feat_rgb = ((line_feat_mat - line_feat_min) / line_feat_max * 255.0).astype(np.int32)
print(line_feat_rgb)
utils.npy2ply(voxels_filteredby_height, line_feat_rgb, "./data/pipeline_height_filtration_vis.ply")

[51.03776573 51.10588386 14.84088054]
[0. 0. 0.]
[[  0   0   0]
 [  7   9  23]
 [  9  27  51]
 ...
 [ 15  51  55]
 [  9  55  98]
 [  8  65 133]]


In [7]:
line_mask = (line_feat_mat[:, 0] > line_feat_mat[:, 0].mean()) & (line_feat_mat[:, 1] > line_feat_mat[:, 1].mean())
# 获取电力线点云
voxels_line = voxels_filteredby_height[line_mask]
colors_line = colors_filteredby_height[line_mask]
utils.npy2ply(voxels_line, colors_line, "./data/pipeline_line.ply")

# Tower extraction

In [9]:
line_excluded_mask = ~line_mask & (line_feat_mat[:, 0] < line_feat_mat[:, 0].mean()) & (line_feat_mat[:, 1] < line_feat_mat[:, 1].mean())
voxels_line_excluded = voxels_filteredby_height[line_excluded_mask]
colors_line_excluded = colors_filteredby_height[line_excluded_mask]
utils.npy2ply(voxels_line_excluded, colors_line_excluded, "./data/pipeline_line_excluded.ply")

In [10]:
if os.path.exists("./data/checkpoint/tower_feat_mat.pkl"):
    with open("./data/checkpoint/tower_feat_mat.pkl", "rb") as f:
        tower_feat_mat = pickle.load(f)
else:
    num_workers = 4
    batch_size = voxels_line_excluded.shape[0] // num_workers
    tower_feat_mat_list = [None] * num_workers
    from concurrent.futures import ProcessPoolExecutor, as_completed
    with ProcessPoolExecutor(max_workers=num_workers) as executor:
        futures_dict = {
            executor.submit(
                utils.eigval_vertic,
                voxels_line_excluded[i * batch_size:(i + 1) * batch_size],
                voxel_size * 75.0
            ) : i for i in range(num_workers)
        }

        for future in as_completed(futures_dict):
            feat_mat, _ = future.result()
            tower_feat_mat_list[futures_dict[future]] = feat_mat
    tower_feat_mat = np.concatenate(tower_feat_mat_list, axis=0)
    with open("./data/checkpoint/tower_feat_mat.pkl", "wb") as f:
        pickle.dump(tower_feat_mat, f)

eigval progress: 100%|██████████████████████████████████████| 76401/76401 [01:12<00:00, 1047.95it/s]


In [20]:
# 也许这里会损失一些额外的点云，由于前面的处理方法的遗留问题
voxels_line_excluded = voxels_line_excluded[:len(tower_feat_mat)]
colors_line_excluded = colors_line_excluded[:len(tower_feat_mat)]

tower_feat_max = np.max(tower_feat_mat, axis=0)
tower_feat_min = np.min(tower_feat_mat, axis=0)
tower_feat_rgb = ((tower_feat_mat - tower_feat_min) / tower_feat_max * 255.0).astype(np.int32)
utils.npy2ply(voxels_line_excluded, tower_feat_rgb, "./data/pipeline_line_excluded_vis.ply")

tower_feat_mat = (tower_feat_mat - tower_feat_min) / tower_feat_max # normalized to [0, 1]
half_max_height = np.median(tower_feat_mat[:, 1])
tower_mask = \
    (tower_feat_mat[:, 0] > 0.25) & \
    (tower_feat_mat[:, 1] > half_max_height) & \
    (tower_feat_mat[:, 2] < 0.1)
print(tower_mask.astype(np.int32).sum())
voxels_tower = voxels_line_excluded[tower_mask]
colors_tower = colors_line_excluded[tower_mask]
utils.npy2ply(voxels_tower, colors_tower, "./data/pipeline_tower.ply")

79207


In [23]:
# denoised 降噪
labels = np.array(utils.npy2o3d(voxels_tower).cluster_dbscan(eps=voxel_size * 75.0, min_points=100, print_progress=False))
if labels.shape[0] != voxels_tower.shape[0]:
    raise ValueError("mismatched label num")

print(f"number of clustered towers: {labels.max() + 1}")
unique_labels, labels_count = np.unique(labels, return_counts=True)

unique_labels = unique_labels[1:]
labels_count = labels_count[1:]
min_threshold = labels_count.mean() // 2
label_selection = labels_count > min_threshold
valid_labels = unique_labels[label_selection]
print(f"tower point threshold: {min_threshold}")

print(f"valid labels: {valid_labels}")
tower_denoised_mask = [(labels == label_idx) for label_idx in valid_labels]
tower_denoised_mask = np.logical_or.reduce(
    np.vstack(tower_denoised_mask),
    axis=0
)
voxels_tower_denoised = voxels_tower[tower_denoised_mask]
colors_tower_denoised = colors_tower[tower_denoised_mask]

utils.npy2ply(
        voxels_tower_denoised,
        colors_tower_denoised,
    "./data/pipeline_tower_denoised.ply"
)

number of clustered towers: 11
tower point threshold: 3588.0
valid labels: [ 0  1  2  3  5  6  7  8  9 10]


In [24]:
instanced_labels = utils.cluster_instanced(voxels_tower_denoised, voxel_size * 200.0, min_threshold)
print(f"number of cluster: {instanced_labels.max()}")
voxels_tower_instanced = voxels_tower_denoised
colors_tower_instanced = np.vstack([np.array([0, 0, 0])] * len(voxels_tower_instanced))
for label in range(1, instanced_labels.max() + 1):
    print(f"tower#{label}: {(instanced_labels == label).astype(np.int32).sum():05d}")
    colors_tower_instanced[instanced_labels == label] = np.array([
        np.random.randint(100, 255),
        np.random.randint(100, 255),
        np.random.randint(100, 255)
    ])
utils.npy2ply(voxels_tower_instanced, colors_tower_instanced, "./data/pipeline_tower_instanced.ply")


number of cluster: 10
tower#1: 07304
tower#2: 15903
tower#3: 05230
tower#4: 08318
tower#5: 08530
tower#6: 05297
tower#7: 08097
tower#8: 07507
tower#9: 07592
tower#10: 03610


# Segmentation using line's and tower's position

In [25]:
print(f"number of voxels in line:\t{len(voxels_line)}")
print(f"number of voxels in tower:\t{len(voxels_tower)}")

def position_instanced(points: np.ndarray, poses: np.ndarray, radius: float, mask: int):
    import open3d as o3d
    
    o3d_pcd = utils.npy2o3d(points)
    search_tree = o3d.geometry.KDTreeFlann(o3d_pcd)

    instanced_labels = np.zeros((len(points), ), dtype=np.int32)

    for query in poses:
        neighbour_num, neighbour_indicies, _ = search_tree.search_radius_vector_3d(query, radius)
        instanced_labels[neighbour_indicies] = mask
    
    return instanced_labels

labels_line = position_instanced(voxels, voxels_line, voxel_size * 20.0, 1)
labels_tower = position_instanced(voxels, voxels_tower_instanced, voxel_size * 50.0, 2)

print(f"number of label in line:\t{labels_line.astype(np.int32).sum()}")
print(f"number of label in tower:\t{labels_tower.astype(np.int32).sum()}")


# give different colors to line and tower
colors_labeled = colors
colors_labeled[labels_line > 0] = np.array([255, 255, 100])
colors_labeled[labels_tower > 0] = np.array([100, 255, 255])

utils.npy2ply(voxels, colors_labeled, "./data/pipeline_final.ply")

number of voxels in line:	86354
number of voxels in tower:	79207
number of label in line:	138810
number of label in tower:	227624
